#### Names of people in the group

Please write the names of the people in your group in the next cell.

Jacob Ørmen Clements

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
# Loading modules that we need
from pyspark.sql.dataframe import DataFrame
from collections import Counter
from pyspark.sql.functions import desc
#These are libraries that I found useful for solving the problems that I needed to import
from pyspark.sql.functions import avg, col, regexp_replace, split, size, array, lit, explode, posexplode, when, size
import math

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame that I have altered to work with my code from task 1
def load_df(table_name: "name of the table to load") -> DataFrame:
    # Match the path with the one used in the save_df function
    path = f"dbfs:/user/hive/warehouse/{table_name}.parquet"
    return spark.read.parquet(path)

users_df = load_df("users")
posts_df = load_df("posts")

#### Subtask 1: implementing two functions
Implement these two functions:
1. 'compute_pearsons_r' that receives a DataFrame and two column names and returns the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between values of two columns;
2. 'make_tag_graph' that in the input receives the DataFrame containing the records related to 'questions' and returns a DataFrame with two columns 'u' and 'v'; the record for row i from the resulting DataFrame is a tuple (u_i, v_i). u_i and v_j are distinct tags and have appeared together for a question.

Please note that you should implement the 'compute_pearsons_r' yourself, so you should not use the 'DataFrame.stat.corr' method. Nevertheless, you can use 'DataFrame.stat.corr' to verify the correctness of your implementation.

In [0]:
def compute_pearsons_r(df: "a DataFrame", col1: "name of column A", col2: "name of column B") -> float:
# Selects the two columns from the DataFrame df and applies the avg function to each, .first() returns the row of averages we calculated
    avg_col1, avg_col2 = df.select(avg(col1), avg(col2)).first() 

    # Creates new columns of differences, where alias renames the columns
    df = df.select(
        (col(col1) - avg_col1).alias('col1_diff'),
        (col(col2) - avg_col2).alias('col2_diff')
    )
    #Here we calculate the values we need to calcualte Pearson's r, where the agg function sums up the resulting columns of the operations
    # .first()[0] fetches the value of the values calculated
    numerator = df.select((col('col1_diff') * col('col2_diff')).alias('product')).agg({'product': 'sum'}).first()[0]
    denominator_col1 = df.select((col('col1_diff') ** 2).alias('col1_sq')).agg({'col1_sq': 'sum'}).first()[0]
    denominator_col2 = df.select((col('col2_diff') ** 2).alias('col2_sq')).agg({'col2_sq': 'sum'}).first()[0]
    
    # Compute Pearson's r which is basically just regular Python code
    r = numerator / (math.sqrt(denominator_col1) * math.sqrt(denominator_col2))
    
    return r

# This was difficult, quite long comments when making sure I knew what happened in each line.
# In the function I have comments on what each line intends to do, whilst I have some comments on what the functions do under the implementation
def make_tag_graph(df: "DataFrame containing question data") -> DataFrame:
    # First, clean the 'Tags' column by removing "<>"-brackets
    # Then, split the cleaned 'Tags' string into an array where each element is a tag. 
    # After code we can analyse individual tags which were grouped together as string.
    tags_array = split(regexp_replace(col("Tags"), "^<|>$", ""), "><")
    
    # In cases where there's only one tag, create array with the tag and null value.
    df = df.withColumn("TagsArray", when(size(tags_array) == 1, array(regexp_replace(col("Tags"), "[<>]", ""), lit(None))).otherwise(tags_array))

    # Transform each element of an array into a separate row, effectively multiplying the rows by the number of tags
    exploded_tags = df.withColumn("tag", explode("TagsArray")).select("Id", "tag")
    
    # Creating aliases for the exploded DataFrame to facilitate a self-join. 
    # This self-join will allow us to create pairs of tags that appear together.
    df1_alias = exploded_tags.alias("df1")
    df2_alias = exploded_tags.alias("df2")

    # Performing a self-join on the 'Id' column to pair tags from the same question.
    joined_df = df1_alias.join(df2_alias, "Id")

    # Filtering the self-joined DataFrame to get unique pairs of tags. 
    # The filter conditions ensure that we exclude duplicate tag pairs and include (tag, null) pairs 
    # for questions with a single tag, ensuring they are represented in the graph.
    tag_pairs = joined_df.select(
        col("df1.tag").alias("u"),
        col("df2.tag").alias("v")
    ).filter((col("u") != col("v")) | (col("u").isNotNull() & col("v").isNull()))

    return tag_pairs

# regexp_replace: Replaces all occurrences of a pattern in a string with a replacement string.
# split: Splits a string into an array of substrings based on a delimiter.
# withColumn: Adds a new column or replaces an existing column with the specified expression.
# when: Evaluates a list of conditions and returns one of multiple possible result expressions.
# size: Returns the number of elements in an array.
# array: Creates a new array column with the input columns or expressions.
# lit: Creates a Column of literal value.
# otherwise: A default expression for "when" if none of the conditions are met.
# explode: Creates a new row for each element in the given array or map column.
# join: Joins with another DataFrame, using the given join expression.
# filter: Filters rows using the given condition, which is an expression that evaluates to true or false.


#### Subtask 2: implementing three functions
Impelment these three functions:
1. 'get_nodes' that, given the result from execution of 'make_tag_graph', returns a DataFrame with one column named 'id' that includes the tags that have appeared in the tag graph;
2. 'get_edges' that, given the result from execution of 'make_tag_graph', returns a DataFrame with two columns 'src' and 'dst' where 'src' is the source node and 'dst' is the destination node.


Note that the term 'tag graph' in this context refers to the DataFrame reuturned by executing 'make_tag_graph'. Furthermore, 'src' and 'dst' are distinct, so 'src' != 'dst'.

In [0]:
def get_nodes(df: "DataFrame of the tag graph") -> DataFrame:
    # Select distinct u tags and rename the column to id
    u_nodes = df.select(col("u").alias("id")).distinct()
    
    # Select distinct v tags and rename the column to id
    v_nodes = df.select(col("v").alias("id")).distinct()

    # Combine both u and v tags, remove nulls, and get distinct values
    all_nodes = u_nodes.union(v_nodes).filter(col("id").isNotNull()).distinct()

    return all_nodes



def get_edges(df: "DataFrame of the tag graph") -> DataFrame:
    # Rename the columns u and v to src and dst
    edges_df = df.select(
        col("u").alias("src"),
        col("v").alias("dst")
    )
    # According to the Piazza note, we don't remove any pair unless both u and v are null
    valid_edges = edges_df.filter("src IS NOT NULL AND dst IS NOT NULL")

    return valid_edges



In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 3: validating the implementation by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
%%unittest_main
class TestTask3(unittest.TestCase):
  
  error_threshold = 0.03
  
  def test_corr1(self):
    # Pearson correlation coefficient between 'user reputation' and 'upvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "UpVotes")
    self.assertLessEqual(abs(result-0.5218138310114108), self.error_threshold)
    print(result)
  
  def test_corr2(self):
    # Pearson correlation coefficient between 'user reputation' and 'downvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "DownVotes")
    self.assertLessEqual(abs(result-0.1473558141546844), self.error_threshold)
    print(result)

  def test_corr3(self):
    # Pearson correlation coefficient between 'question score' and the 'number of answers' it received
    result = compute_pearsons_r(posts_df[posts_df["PostTypeId"] == 1], "Score", "AnswerCount")
    self.assertLessEqual(abs(result-0.47855272641249674), self.error_threshold)
    print(result)
    
  def test_make_tag_graph(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    self.assertIsInstance(result, DataFrame)
    
    coulmn_names = Counter(map(str.lower, ['u', 'v']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)), "Missing column(s) or column name mismatch")
    
    display(result)
    
    self.assertEqual(result.count(), 228830)
    
  def test_get_nodes(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    n = get_nodes(result)
    self.assertEqual(n.count(), 638)
    n.show()

  def test_get_edges(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    e = get_edges(result)
    
    coulmn_names = Counter(map(str.lower, ['src', 'dst']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, e.columns)), "Missing column(s) or column name mismatch")
    
    self.assertEqual(e.count(), 225290)
    e.show()
    

0.5218136972418368
0.14735577823040416
0.47530986340467524
+------------------+------------------+
|               src|               dst|
+------------------+------------------+
|  machine-learning|    classification|
|    classification|  machine-learning|
|       experiments|         marketing|
|         marketing|       experiments|
|          sampling|     preprocessing|
|     preprocessing|          sampling|
|feature-extraction|          sequence|
|          sequence|feature-extraction|
|  machine-learning| bayesian-networks|
| bayesian-networks|  machine-learning|
|      scikit-learn|           xgboost|
|      scikit-learn|            pandas|
|            pandas|           xgboost|
|            pandas|      scikit-learn|
|           xgboost|            pandas|
|           xgboost|      scikit-learn|
|            search| audio-recognition|
| audio-recognition|            search|
|  machine-learning|          azure-ml|
|  machine-learning|           xgboost|
+------------------+-

u v machine-learning classification classification machine-learning apache-hadoop null experiments marketing marketing experiments sampling preprocessing preprocessing sampling feature-extraction sequence sequence feature-extraction machine-learning bayesian-networks bayesian-networks machine-learning scikit-learn xgboost scikit-learn pandas pandas xgboost pandas scikit-learn xgboost pandas xgboost scikit-learn search audio-recognition audio-recognition search machine-learning azure-ml machine-learning xgboost machine-learning decision-trees machine-learning logistic-regression logistic-regression azure-ml logistic-regression xgboost logistic-regression decision-trees logistic-regression machine-learning decision-trees azure-ml decision-trees xgboost decision-trees logistic-regression decision-trees machine-learning xgboost azure-ml xgboost decision-trees xgboost logistic-regression xgboost machine-learning azure-ml xgboost azure-ml decision-trees azure-ml logistic-regression azure-ml machine-learning time-series descriptive-statistics descriptive-statistics time-series visualization plotting plotting visualization machine-learning r r machine-learning feature-selection cross-validation cross-validation feature-selection data-mining null neural-network convolutional-neural-network neural-network tensorflow tensorflow convolutional-neural-network tensorflow neural-network convolutional-neural-network tensorflow convolutional-neural-network neural-network machine-learning clustering machine-learning r r clustering r machine-learning clustering r clustering machine-learning class-imbalance smote class-imbalance text-classification text-classification smote text-classification class-imbalance smote text-classification smote class-imbalance machine-learning data machine-learning data-cleaning machine-learning dataset machine-learning data-mining data-mining data data-mining data-cleaning data-mining dataset data-mining machine-learning dataset data dataset data-cleaning dataset data-mining dataset machine-learning data-cleaning data data-cleaning dataset data-cleaning data-mining data-cleaning machine-learning data data-cleaning data dataset data data-mining data machine-learning scikit-learn lda scikit-learn topic-model scikit-learn multiclass-classification scikit-learn svm svm lda svm topic-model svm multiclass-classification svm scikit-learn multiclass-classification lda multiclass-classification topic-model multiclass-classification svm multiclass-classification scikit-learn topic-model lda topic-model multiclass-classification topic-model svm topic-model scikit-learn lda topic-model lda multiclass-classification lda svm lda scikit-learn machine-learning time-series time-series machine-learning python decision-trees python scikit-learn scikit-learn decision-trees scikit-learn python decision-trees scikit-learn decision-trees python python training python pandas python predictive-modeling python scikit-learn scikit-learn training scikit-learn pandas scikit-learn predictive-modeling scikit-learn python predictive-modeling training predictive-modeling pandas predictive-modeling scikit-learn predictive-modeling python pandas training pandas predictive-modeling pandas scikit-learn pandas python training pandas training predictive-modeling training scikit-learn training python feature-selection feature-engineering feature-engineering feature-selection probability null neural-network cnn cnn neural-network r glm glm r experiments null python class-imbalance python scikit-learn scikit-learn class-imbalance scikit-learn python class-imbalance scikit-learn class-imbalance python machine-learning numerical machine-learning activation-function machine-learning backpropagation machine-learning deep-learning deep-learning numerical deep-learning activation-function deep-learning backpropagation deep-learning machine-learning backpropagation numerical backpropagation activation-function backpropagation deep-learning backpropagation machine-l

Success

......
----------------------------------------------------------------------
Ran 6 tests in 33.234s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please write a short description for the terms below---one to two short paragraphs for each term. Don't copy-paste; instead, write your own understanding.

1. What do the terms 'User-Defined Functions (UDFs)', 'Data Locality', 'Bucketing', 'Distributed Filesystem' mean in the context of Spark?

Write your descriptions in the next cell.

# Answers
### User-Defined Functions (UDFs)
User-Defined Functions (UDFs) in Spark are functions implemented to allow users to extend the standard functionalities of its SQL and DataFrames. UDFs help in implementing reusable code for operations that do not find a place in the built-in functions of the API of Spark. UDFs are column-wise operations on DataFrames, and they are mainly useful for data transformation or complex computation. Nevertheless, they should be used in a balanced manner, as the optimizer of Spark cannot optimize them, and this might bring in performance issues.

### Data Locality:
Data Locality in Spark implies the data being nearer to the computation resources executing the tasks on that data. It schedules tasks on nodes where the data is already present, therefore minimizing the cost of I/O on the network. There are different levels of data locality, such as PROCESS_LOCAL (data is in the memory of the same process), NODE_LOCAL (data is on the same node), NO_PREF (no preference of data location) etc. which ensures data locality improves performance by reducing the time spent on moving data across the network.

### Bucketing
Bucketing is a concept in Spark where one specifies the data that will be separated into manageable and comprehensible portions known as "buckets." It is particularly known for better optimization of join operations in Spark where data is distributed into pre-known divisions governed by a given column or a set of columns. Bucketing can help in such a way that it gives Spark the possibility to optimize joins, hence making them as fast as possible by preventing unnecessary data shuffling across partitions. In fact, bucketing can significantly boost some operations and improve the performance of the queries.

### Distributed Filesystem:
In Spark, a Distributed Filesystem refers to a storage system where data is stored on multiple machines in a distributed manner, while the user still views the same data as a single coherent system. For instance, Hadoop Distributed File System (HDFS) and Amazon S3. These file systems are designed to handle a large volume of data, hence offering high throughput access. Spark is integrated with these to read and write data parallely in cluster, which means big data can be easily managed by it. Some added resiliency and fault tolerance for Spark comes due to the fact that these file systems are distributed, can handle node failures, and ensure data availability.
